In [2]:
import pandas as pd
import sklearn.model_selection
from sklearn.model_selection import KFold

In [6]:
df = pd.read_csv("../data/processed/test_train.csv").drop(columns = ['Unnamed: 0'])
df

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_is_S2,flag_is_S0,flag_is_S3,flag_is_RSTO,flag_is_RSTR,flag_is_RSTOS0,flag_is_OTH,flag_is_SH,normal,test
0,0,181,5450,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,4.0
1,0,239,486,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,3.0
2,0,235,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,4.0
3,0,219,1337,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,4.0
4,0,217,2032,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,310,1881,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,2.0
494017,0,282,2286,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,1.0
494018,0,203,1200,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,4.0
494019,0,291,1200,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,True,2.0


## Collation of attacks into the 4 main categories

Now that we have split up the different attacks into categories, we will now combine these into the 4 main attack categories. To do this I will run through each of the attacks and work out which category each belongs to. Here we have the categories and the different attack types.

* DoS - back, land, neptune, pod, smurf, teardrop
* R2L - ftp_write, guess_passwd, imap, multihop, phf, spy, warezclient, warezmaster
* U2R - buffer_overflow, loadmodule, perl, rootkit
* Probing - nmap, ipsweep, portsweep, satan

So whenever there is a 1 in the type column we will add a 1 to the corresponding category. E.g. if df[i,'type_is_back.'] == 1 then we set df[i,'is_DoS'] = 1.

The following code was a first implementation of separating out the attack types into the attack categories. I then decided I could skip this step and immediately look at the implementation of the Test column.

In [ ]:
#Unused code
for i in range(df.shape[0]):
    if (df.at[i,'type_is_back.'] == 1) or (df.at[i,'type_is_land.'] == 1) or (df.at[i,'type_is_neptune.'] == 1) or (df.at[i,'type_is_pod.'] == 1) or (df.at[i,'type_is_smurf.'] == 1) or (df.at[i,'type_is_teardrop.'] == 1):
        df.at[i,'category_is_DoS'] = 1
    else:
        df.at[i,'category_is_DoS'] = 0
    if (df.at[i,'type_is_ftp_write.'] == 1) or (df.at[i,'type_is_guess_passwd.'] == 1) or (df.at[i,'type_is_imap.'] == 1) or (df.at[i,'type_is_multihop.'] == 1) or (df.at[i,'type_is_phf.'] == 1) or (df.at[i,'type_is_spy.'] == 1) or (df.at[i,'type_is_warezclient.'] == 1) or (df.at[i,'type_is_warezmaster.'] == 1):
        df.at[i,'category_is_R2L'] = 1
    else:
        df.at[i,'category_is_R2L'] = 0
    if (df.at[i,'type_is_buffer_overflow.'] == 1) or (df.at[i,'type_is_loadmodule.'] == 1) or (df.at[i,'type_is_perl.'] == 1) or (df.at[i,'type_is_rootkit.'] == 1):
        df.at[i,'category_is_U2R'] = 1
    else:
        df.at[i,'category_is_U2R'] = 0
    if (df.at[i,'type_is_nmap.'] == 1) or (df.at[i,'type_is_ipsweep.'] == 1) or (df.at[i,'type_is_portsweep.'] == 1) or (df.at[i,'type_is_satan.'] == 1):
        df.at[i,'category_is_Probe'] = 1
    else:
        df.at[i,'category_is_Probe'] = 0
    if (df.at[i,'type_is_normal.'] == 1):
        df.at[i,'category_is_Normal'] = 1
    else:
        df.at[i,'category_is_Normal'] = 0

In [4]:
for i in range(df.shape[0]):
    if (df.at[i,'type_is_back.'] == 1) or (df.at[i,'type_is_land.'] == 1) or (df.at[i,'type_is_neptune.'] == 1) or (df.at[i,'type_is_pod.'] == 1) or (df.at[i,'type_is_smurf.'] == 1) or (df.at[i,'type_is_teardrop.'] == 1):
        df.at[i,'normal'] = False
        df.at[i,'test'] = 1
    elif (df.at[i,'type_is_ftp_write.'] == 1) or (df.at[i,'type_is_guess_passwd.'] == 1) or (df.at[i,'type_is_imap.'] == 1) or (df.at[i,'type_is_multihop.'] == 1) or (df.at[i,'type_is_phf.'] == 1) or (df.at[i,'type_is_spy.'] == 1) or (df.at[i,'type_is_warezclient.'] == 1) or (df.at[i,'type_is_warezmaster.'] == 1):
        df.at[i,'normal'] = False
        df.at[i,'test'] = 2
    elif (df.at[i,'type_is_buffer_overflow.'] == 1) or (df.at[i,'type_is_loadmodule.'] == 1) or (df.at[i,'type_is_perl.'] == 1) or (df.at[i,'type_is_rootkit.'] == 1):
        df.at[i,'normal'] = False
        df.at[i,'test'] = 3
    elif (df.at[i,'type_is_nmap.'] == 1) or (df.at[i,'type_is_ipsweep.'] == 1) or (df.at[i,'type_is_portsweep.'] == 1) or (df.at[i,'type_is_satan.'] == 1):
        df.at[i,'normal'] = False
        df.at[i,'test'] = 4
    elif (df.at[i,'type_is_normal.'] == 1):
        df.at[i,'normal'] = True
        df.at[i,'test'] = 0

We will now check that the code has worked by looking only at the 'type_is_' and 'category_is_' columns for not the attack entries.

In [5]:
#Code Testing
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,type_is_back.,type_is_imap.,type_is_satan.,type_is_phf.,type_is_nmap.,type_is_multihop.,type_is_warezmaster.,type_is_warezclient.,type_is_spy.,type_is_rootkit.
0,0,tcp,http,SF,181,5450,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,tcp,http,SF,239,486,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,tcp,http,SF,235,1337,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,tcp,http,SF,219,1337,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,tcp,http,SF,217,2032,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#Code Testing
checker = df[df['type_is_normal.'] == 0] 
checker.filter(regex=('^(test|type)'))

KeyError: 'type_is_normal.'

We can see that the code works as expected. 

The following was used to find the loaction of the columns to be removed from the dataframe

In [6]:
#Code Testing
df.columns.get_loc('type_is_normal.')

122

In [7]:
#Code Testing
df.columns.get_loc('type_is_rootkit.')

144

We will now remove the 'type_is_\*' columns as we now that we have the 'test' and 'normal' columns.

In [8]:
df.drop(df.iloc[:,122:144], axis=1, inplace = True)

The following section is to test doing a k-fold on the training dataset for each of the main test/training datasets. 

In [8]:
df = pd.read_csv("../data/processed/test_train.csv").drop(columns = ['Unnamed: 0'])
df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,flag_is_S0,flag_is_S3,flag_is_RSTO,flag_is_RSTR,flag_is_RSTOS0,flag_is_OTH,flag_is_SH,type_is_rootkit.,normal,test
0,0,tcp,http,SF,181,5450,0,0,0,0,...,0,0,0,0,0,0,0,0,True,0.0
1,0,tcp,http,SF,239,486,0,0,0,0,...,0,0,0,0,0,0,0,0,True,0.0
2,0,tcp,http,SF,235,1337,0,0,0,0,...,0,0,0,0,0,0,0,0,True,0.0
3,0,tcp,http,SF,219,1337,0,0,0,0,...,0,0,0,0,0,0,0,0,True,0.0
4,0,tcp,http,SF,217,2032,0,0,0,0,...,0,0,0,0,0,0,0,0,True,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494016,0,tcp,http,SF,310,1881,0,0,0,0,...,0,0,0,0,0,0,0,0,True,0.0
494017,0,tcp,http,SF,282,2286,0,0,0,0,...,0,0,0,0,0,0,0,0,True,0.0
494018,0,tcp,http,SF,203,1200,0,0,0,0,...,0,0,0,0,0,0,0,0,True,0.0
494019,0,tcp,http,SF,291,1200,0,0,0,0,...,0,0,0,0,0,0,0,0,True,0.0
